# Notebook #3: Queries, Visualización y Análisis

- En este tercer notebook realizaremos consultas sobre nuestra base de datos y las usaremos para generar visualizaciones y el análisis de las mismas.

- Las visualizaciones de Ploty no se renderizan en Github. Para poder verlas es necesaria la descarga del Notebook.

- A lo largo el notebook, los números con el formato "(1)" representan fuentes externas de información utilizadas, que se referencian al final de este notebook.

- El primer paso será importar las librerías necesarias:

In [3]:
# Librerías para tratamiento de datos
import pandas as pd
pd.set_option("display.max_columns", None) # Parámetro que modifica la visualización de los resultado3s
import numpy as np

# Trabajar con bases de datos SQL
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

# Librería para el acceso a variables y funciones
import sys
sys.path.append("../")
from src import soporte_funciones as sf #Archivo .py donde encontraremos todas nuestras funciones.

# Librería de mapas
import folium

# Librerías para graficar
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Librería para ignorar avisos
import warnings
warnings.filterwarnings("ignore") # Ignora TODOS los avisos

### Objeto del Análisis 

- Antes de plantearnos las queries que pretendemos realizar, debemos recordar que el objetivo del proyecto es obtener insights sobre el mercado, así como la estructura de los datos.

- La información con la que contamos tiene relación con el mercado de la vivienda en Madrid, e incluye:
    - Datos de viviendas turísticas (AirBnB).
    - Datos de viviendas de alquiler (Idelista y Redpiso).
    - Datos sobre los ingresos por hogar (INE).
    - Datos sobre cantidad de población nacional y extranjera (Ayuntamiento de Madrid).

#### Diagrama Entidad Relación

<img src="../images/DiagramaER.png" width="400">

- Para ejecutar nuestras queries, haremos uso de la función `sf.dbeaver_fetch()`, que recibe como argumentos la conexión a DBeaver (usamos la función `sf.dbeaver_conexion()`) y la query.

### 1. Precio Promedio del Alquiler por Distrito

In [ ]:
query1= '''
SELECT d.nombre, 
    round(AVG(i.precio),2) AS avg_precio_idealista, 
    round(AVG(r.precio),2) AS avg_precio_redpiso 
FROM distritos d
LEFT JOIN idealista i ON d.id_distrito = i.id_distrito
LEFT JOIN redpiso r ON d.id_distrito = r.id_distrito
LEFT JOIN airbnb a ON d.id_distrito = a.id_distrito
GROUP BY d.nombre
ORDER BY avg_precio_idealista DESC, avg_precio_redpiso DESC
;'''

In [16]:
resultado1 = sf.dbeaver_fetch(sf.dbeaver_conexion("alquileresmadrid"),query1)
resultado1

,nombre,avg_precio_idealista,avg_precio_redpiso
0,Salamanca,3382.30,1377.27
1,Hortaleza,3129.55,1167.68
2,Chamartin,2583.06,1211.45
3,Retiro,2444.25,2027.65
4,Centro,2424.74,1088.95
5,Chamberi,2299.35,1367.30
6,Moncloa-Aravaca,2217.93,1225.00
7,Tetuan,2125.40,1161.00
8,Fuencarral-El Pardo,1788.85,1133.40
9,Arganzuela,1734.50,1121.43


### 2. Porcentaje del Ingreso del Hogar dedicado al Alquiler

In [63]:
query2='''
SELECT d.nombre, 
    round(AVG(i.precio)+AVG(r.precio)/2,2) AS alquiler_prom, 
    AVG(ih.total)/12 AS ingreso_prom_hogar,
    (AVG(i.precio)+AVG(r.precio)/2) / (AVG(ih.total)/12) AS porc_alquiler_ingreso
FROM distritos d
LEFT JOIN idealista i ON d.id_distrito = i.id_distrito
LEFT JOIN redpiso r ON d.id_distrito = r.id_distrito
LEFT JOIN airbnb a ON d.id_distrito = a.id_distrito
LEFT JOIN ingresos_hogar ih ON d.id_distrito = ih.id_distrito
GROUP BY d.nombre
ORDER BY porc_alquiler_ingreso DESC
;'''


In [64]:
resultado2 = sf.dbeaver_fetch(sf.dbeaver_conexion("alquileresmadrid"),query2)
resultado2

,nombre,alquiler_prom,ingreso_prom_hogar,porc_alquiler_ingreso
0,Villa de Vallecas,None,2769.604167,NaN
1,Centro,2969.22,2999.677083,0.989846
2,Hortaleza,3713.39,4343.520833,0.854925
3,Tetuan,2705.90,3172.885417,0.852820
4,Salamanca,4070.93,4961.343750,0.820530
5,Retiro,3458.07,4637.322917,0.745705
6,Usera,1621.43,2349.500000,0.690116
7,Puente de Vallecas,1524.30,2256.322917,0.675569
8,Latina,1800.28,2698.114583,0.667237
9,Chamberi,2983.00,4520.770833,0.659844


### 4. Conteo de Anuncios de Alquiler por Distrito

In [78]:
query4 = '''
SELECT d.nombre, 
       (SELECT COUNT(*) FROM idealista i WHERE i.id_distrito = d.id_distrito) AS count_idealista, 
       (SELECT COUNT(*) FROM redpiso r WHERE r.id_distrito = d.id_distrito) AS count_redpiso, 
       (SELECT COUNT(*) FROM airbnb a WHERE a.id_distrito = d.id_distrito) AS count_airbnb
FROM distritos d;
;'''

In [79]:
resultado4 = sf.dbeaver_fetch(sf.dbeaver_conexion("alquileresmadrid"),query4)
resultado4

,nombre,count_idealista,count_redpiso,count_airbnb
0,Centro,78,62,210
1,Arganzuela,20,35,29
2,Retiro,20,17,8
3,Salamanca,37,33,13
4,Chamartin,36,69,4
5,Tetuan,25,25,22
6,Chamberi,31,74,26
7,Fuencarral-El Pardo,13,47,1
8,Moncloa-Aravaca,28,26,6
9,Latina,16,31,3


### 5. Cambio de la población en el período del 2018 al 2022

In [76]:
query5='''
SELECT d.nombre, 
       SUM(p.espanioles) AS total_espanoles, 
       SUM(p.extranjeros) AS total_extranjeros, 
       SUM(p.total) AS total_cambio
FROM distritos d
LEFT JOIN poblacion p ON d.id_distrito = p.id_distrito
GROUP BY d.nombre
ORDER BY total_cambio DESC
;'''

In [77]:
resultado5 = sf.dbeaver_fetch(sf.dbeaver_conexion("alquileresmadrid"),query5)
resultado5

,nombre,total_espanoles,total_extranjeros,total_cambio
0,Hortaleza,6955,5965,12918
1,Carabanchel,-2868,14443,11574
2,Villaverde,1811,9439,11246
3,Villa de Vallecas,5306,5231,10543
4,Vicalvaro,6419,2820,9239
5,Puente de Vallecas,-5441,13529,8087
6,Centro,-1425,9189,7761
7,Fuencarral-El Pardo,3201,4434,7634
8,Usera,-1834,7862,6031
9,San Blas,-2606,7116,4509
